In [94]:
import requests, bs4, selenium
from selenium import webdriver
link = ["https://ocw.mit.edu/courses/11-s940-development-planning-and-implementation-the-dialectic-of-theory-and-practice-fall-2015/", "https://ocw.mit.edu/courses/11-s955-the-sustainability-response-to-covid-19-january-iap-2021/", "https://ocw.mit.edu/courses/12-007-geobiology-spring-2013/", "https://ocw.mit.edu/courses/16-810-engineering-design-and-rapid-prototyping-january-iap-2007/"]

In [97]:
PATH = './driver/chromedriver.exe'
driver = webdriver.Chrome(PATH)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
}
for i in link:
    driver.get(i)
    html = driver.page_source
    d = bs4.BeautifulSoup(html, "html.parser").find_all("div", {"id" : "course-description"})[0].text
    lvl = bs4.BeautifulSoup(html, "html.parser").find_all("a", {"class" : "course-info-level"})[0].text
    tit = bs4.BeautifulSoup(html, "html.parser").find_all("a", {"class" : "text-uppercase display-4 font-weight-bold m-0 text-white"})[0].text
    idl = bs4.BeautifulSoup(html, "html.parser").find_all("ul", {"class" : "list-unstyled m-0"})[1]
    t = bs4.BeautifulSoup(html, "html.parser").find_all("table", {"id" : "course-info-table"})
    for c in t:
        for a in c.find_all("ul", {"class" : "list-unstyled pb-2 m-0"}):
            for g in a.find_all("li"):
                print(g.text.replace("\n", "").replace ("      >          ", ">"))

C:\Users\mirco\AppData\Local\Temp/ipykernel_11564/742569376.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Social Science>Public Administration
Social Science>Sociology>Community Development
Social Science>Urban Studies
Social Science>Urban Studies>Regional Planning
Social Science>Urban Studies>Urban Planning
Energy>Transportation
Health and Medicine>Public Health
Social Science>Urban Studies>Transportation Planning
Science>Earth Science>Geobiology
Engineering>Systems Engineering>Computational Modeling and Simulation
Engineering>Mechanical Engineering>Mechanical Design
Engineering>Systems Engineering>Systems Optimization
Engineering>Aerospace Engineering>Aerodynamics
Engineering>Aerospace Engineering>Structural Mechanics


In [80]:
class CourseInfo:
    nome=""
    idN=""
    link=""
    difficulty=""
    min_age=0
    max_age=0
    desc=""
    discip={}